In [130]:
import warnings
import pandas 
pandas.set_option('mode.use_inf_as_na', True)

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()

In [6]:
# adata.var_keys.

In [7]:
# result = []
# json = []
# for row in adata.obs.values:
#     result.append([item for kev, item in enumerate(row) ])
# for row in result: json.append({k: v for k, v in zip(adata.obs.columns.tolist(), row)})

# print(json)
#adata.var.highly_variable.keys

In [146]:
adata.var

,name
index,
AL627309.1,AL627309.1
RP11-206L10.2,RP11-206L10.2
RP11-206L10.9,RP11-206L10.9
FAM87B,FAM87B
LINC00115,LINC00115
...,...
C21orf58,C21orf58
PCNT,PCNT
DIP2A,DIP2A


In [77]:

sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

#hv_genes = filtered_keys = df[df['column_name'] == True].index.tolist()

#normal_genes = filtered_keys = df[df['column_name'] == False].index.tolist()

#print(hv_genes, normal_genes)
hv_genes = (list(adata.var[adata.var['highly_variable'] == True].index))
genes = (list(adata.var_names))

hv_columns = [i for i,val in enumerate(genes) if val in hv_genes]
#hv_genes

In [ ]:
len(hv_columns)

In [ ]:
highly_variable_columns = []


for key, gene in enumerate(normal_genes):
    if gene in hv_genes:
        highly_variable_columns += key

In [ ]:
highly_variable_columns
print(hv_genes, normal_genes)

In [ ]:
#preprocess
#batchcontrol
adata

In [ ]:
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation

#write custom model. get better accuracy

In [11]:
sc.pp.calculate_qc_metrics(adata)
5

5

In [12]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)
    for result in results:
        print('url:', result[0], 'time (s):', result[1])

In [13]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [52]:
# gene_expressed = ["string1", "string2", "string3"]

    
# strings = np.array(gene_expressed)
# #adata.obs['genes_expressed'] = strings
# coordinates = []
cx = adata.X
cx = cx.tocoo()

# #delete non highly variable genes from matrix


# for i,j,v in zip(cx.row, cx.col, cx.data):
#     if i in hv_columns: coordinates += [[i,j,v]]
# for coord in coordinates:
#     coord[1] = adata.var_names[coord[1]]

#print(coordinates)

In [78]:
# rowGeneExpression = {}

# for coordinate in coordinates:
#     row = coordinate[0]
#     gene = coordinate[1]
#     pvalue = coordinate[2]
#     if not row in rowGeneExpression:
#         rowGeneExpression[row] = 0
#     if gene in hv_genes: rowGeneExpression[row] += pvalue
# print(rowGeneExpression)

In [79]:
# hv_genes

In [213]:
#hv_columns

In [148]:
from collections import defaultdict
rowGeneExpression = defaultdict(int)
rowGeneExpression2 = defaultdict(dict)


import math
math.floor
for column in hv_columns:
    for row_id in range(math.floor(float(df.shape[0]))):
        rowGeneExpression[row_id] += adata.X[row_id, column]
        #rowGeneExpression2[row_id][column] = adata.X[row_id, column]


In [149]:
len(rowGeneExpression)

24673

In [185]:
df = adata.obs
dependent_variables =  [rowGeneExpression[row] for row in range(df.shape[0])]
df['geneExpressionCount'] = dependent_variables
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()


In [186]:
import torch.optim as optim


criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [187]:
from torch import tensor

t_dep = tensor(dependent_variables) # pertrubations


In [188]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[  3017.0000,    877.0000,    -27.6404,     14.9666,      9.0000,   1704.0000,    711.0000,   1596.0000],
        [  2481.0000,    713.0000,    -27.4936,     28.9249,      9.0000,   1614.0000,    662.0000,   1586.0000],
        [   703.0000,    337.0000,    -10.4682,     -5.9844,      3.0000,    908.0000,    337.0000,    239.0000],
        [  3420.0000,    850.0000,    -24.3680,     20.4293,      9.0000,   1738.0000,    653.0000,   2014.0000],
        [  3158.0000,   1111.0000,     27.9522,     24.1597,      4.0000,   1857.0000,    928.0000,   1141.0000],
        [  1869.0000,    635.0000,     -0.4702,    -25.3987,      5.0000,   1525.0000,    634.0000,    568.0000],
        [  1142.0000,    436.0000,    -15.9062,     20.0853,      9.0000,   1157.0000,    436.0000,    582.0000],
        ...,
        [   635.0000,    424.0000,     -6.6479,     -5.5475,      3.0000,    882.0000,    423.0000,    111.0000],
        [  1340.0000,    480.0000,      7.7202,     33.3402,      8.0000,  

In [189]:
t_indep.shape[1]

8

In [190]:

n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([ 0.3913,  0.0994,  0.3359,  0.0419, -0.1523, -0.4862, -0.0491, -0.1332])

In [191]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [192]:
preds = (t_indep*coeffs).sum(axis=1)
loss = torch.abs(preds-t_dep).mean()
loss = calc_loss(coeffs, t_indep, t_dep)

loss = torch.abs(preds-t_dep).mean()
loss

tensor(713.1940, dtype=torch.float64)

In [193]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [194]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [195]:
t_indep*coeffs

tensor([[ 0.0999,  0.0316, -0.2114,  0.0146, -0.1370, -0.2751, -0.0246, -0.0364],
        [ 0.0822,  0.0257, -0.2102,  0.0282, -0.1370, -0.2605, -0.0229, -0.0361],
        [ 0.0233,  0.0121, -0.0801, -0.0058, -0.0457, -0.1466, -0.0117, -0.0054],
        [ 0.1132,  0.0306, -0.1863,  0.0199, -0.1370, -0.2806, -0.0226, -0.0459],
        [ 0.1046,  0.0400,  0.2138,  0.0236, -0.0609, -0.2998, -0.0321, -0.0260],
        [ 0.0619,  0.0229, -0.0036, -0.0248, -0.0761, -0.2462, -0.0219, -0.0129],
        [ 0.0378,  0.0157, -0.1216,  0.0196, -0.1370, -0.1868, -0.0151, -0.0133],
        ...,
        [ 0.0210,  0.0153, -0.0508, -0.0054, -0.0457, -0.1424, -0.0146, -0.0025],
        [ 0.0444,  0.0173,  0.0590,  0.0325, -0.1218, -0.2137, -0.0166, -0.0192],
        [ 0.0342,  0.0169,  0.1397,  0.0010, -0.0914, -0.1821, -0.0162, -0.0061],
        [ 0.0701,  0.0295, -0.0884,  0.0025, -0.0609, -0.2694, -0.0276, -0.0121],
        [ 0.0504,  0.0188,  0.1923,  0.0064, -0.0914, -0.2295, -0.0181, -0.0080],
   

In [196]:
t_indep = t_indep / vals

In [197]:
coeffs.requires_grad_()


tensor([ 0.3913,  0.0994,  0.3359,  0.0419, -0.1523, -0.4862, -0.0491, -0.1332], requires_grad=True)

In [198]:
loss = calc_loss(coeffs, t_indep, t_dep)

loss.backward()

In [199]:
coeffs.grad

tensor([    -0.0000,     -0.0001,     -0.0002,     -0.0001,     -0.0645,     -0.0001,     -0.0003,     -0.0000])

In [200]:
#dependent_variables

In [201]:
independent_variables = pd.DataFrame(numerical_values)

In [202]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [203]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
#len(trn_indep),len(val_indep)

In [204]:
def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [205]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [206]:
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()
def init_coeffs(): return (torch.rand(n_coeff, 1)*0.1).requires_grad_()


In [207]:
def train_model(epochs=30, lr=0.01):
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [208]:
coeffs = train_model(18, lr=0.2)

633.933; 633.932; 633.932; 633.931; 633.930; 633.929; 633.928; 633.927; 633.927; 633.926; 633.925; 633.924; 633.923; 633.922; 633.922; 633.921; 633.920; 633.919; 

In [209]:
indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT',
'geneExpressionCount'
]    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

{'nCount_RNA': tensor(-0.2832),
 'nFeature_RNA': tensor(0.1566),
 'tsne1': tensor(-0.3726),
 'tsne2': tensor(-0.1476),
 'cluster': tensor(-0.2011),
 'nCount_SCT': tensor(-0.2953),
 'nFeature_SCT': tensor(-0.0597),
 'geneExpressionCount': tensor(-0.3402)}

In [210]:
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_preds(coeffs, indeps): return torch.sigmoid(indeps@coeffs)

calc_preds(coeffs, t_indep)

tensor([0.4965, 0.4962, 0.4991, 0.4962, 0.4961, 0.4980, 0.4958,  ..., 0.4989, 0.4949, 0.4961, 0.4985, 0.4956, 0.4961, 0.4969])

In [211]:
len([prob for prob in calc_preds(coeffs, t_indep) if prob > .5]) # should be 5000

505

In [212]:

acc(coeffs) 

##
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# pass in a h5ad file and get back 
#Predictions:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0]
# Perturbations:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0]

tensor(1.)

In [182]:
print 50
!pip install --quiet hyperopt
!pip install --quiet "ray[tune]"
!pip install --quiet scvi-colab
from scvi_colab import install

install()


import ray
import scanpy as sc
import scvi
from ray import tune
from scvi import autotune

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (517448773.py, line 1)

In [118]:
def benchmarkResults():
    pass

In [119]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)






# adata_t = adata[
#     ~(
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()

# cd4t_stim = adata[
#     (
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()








# scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [120]:
# model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [121]:
# model.train(
#     max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
# )

In [122]:
# adata_t.obsm["scgen"] = model.get_latent_representation()

In [123]:
# sc.pp.neighbors(adata_t, use_rep="scgen")
# sc.tl.umap(adata)

In [ ]:
# adata_t

In [ ]:
# pred, delta = model.predict(
#     ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
# )

# # we annotate the predicted cells to distinguish them later from ground truth cells.
# pred.obs["condition"] = "prediction"
# print(pred)

In [ ]:
# pred.obs

In [ ]:

# train = adata
# ctrl_adata = adata[
#     ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
# ]
# stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# # concatenate pred, control and real CD4 T cells in to one object
# eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [ ]:
# sc.tl.pca(eval_adata)
# sc.pl.pca(eval_adata, color="condition", frameon=False)

In [ ]:
# cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [ ]:
# sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
# diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
# diff_genes

In [ ]:
# r2_value = model.reg_mean_plot(
#     eval_adata,
#     axis_keys={"x": "predicted stimulated", "y": "stimulated"},
#     gene_list=diff_genes[:10],
#     top_100_genes=diff_genes,
#     labels={"x": "predicted", "y": "ground truth"},
#     show=True,
#     legend=False,
# )

In [ ]:
# sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
# import pertpy as pt
# import muon as mu
# import scanpy as sc

In [ ]:
# mdata = pt.dt.papalexi_2021()
# for col in mdata.obs: print(col)

In [ ]:
# sc.pp.normalize_total(mdata["rna"])
# sc.pp.log1p(mdata["rna"])
# sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
# mu.prot.pp.clr(mdata["adt"])

In [ ]:
# sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
# sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
# sc.tl.umap(mdata["rna"])

In [ ]:
# sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)